# Overview

This notebook is the driver to run Anemoi inference as AML job.

## Step 1: Configure local environment

In [ ]:
!az configure --defaults workspace=<your workspace> group=<your Azure resource group>

## Step 2: Create Anemoi Inference AML Environment

In [1]:
%%writefile anemoi_inference_env.yaml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: anemoi_inference
build:
  path: ./environment

Writing anemoi_inference_env.yaml


In [ ]:
!az ml environment create --file anemoi_inference_env.yaml

## Step 3: Submit the job to AML

In [2]:
%%writefile anemoi_job_inference.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./src
command: >-
  python ./anemoi_job_inference.py ${{inputs.model_name}} ${{inputs.input_data}} ${{outputs.output_dir}}
inputs:
  model_name: ecmwf/aifs-single-0.2.1
  input_data: 
    type: uri_folder
    mode: ro_mount
    path: azureml://subscriptions/<your subsription>/resourcegroups/<your resource group>/workspaces/graphcast-workspace/datastores/workspaceblobstore/paths/azureml/<job name>/output_dir/
outputs:
  output_dir:
    type: uri_folder
    mode: rw_mount
services:
    my_vs_code:
      type: vs_code
      nodes: all 
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
compute: azureml:Standard-NC24ads-A100-v4
display_name: anemoi_job_inference
environment: azureml:anemoi_inference@latest
description: Anemoi_Job_Inference

Overwriting anemoi_job_inference.yaml


In [ ]:
!az ml job create -f anemoi_job_inference.yaml